In [1]:
import pandas as pd
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'./src/2year_time/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'./src/2year_time/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open("./corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()

    
def corpus_init():
    ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open("./corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus("./corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [3]:
corpus_init()
tokenizer = return_tokenizer()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1249734 from 2753662 sents. mem=0.333 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9746776, mem=2.907 Gb
[Noun Extractor] batch prediction was completed for 289636 words
[Noun Extractor] checked compounds. discovered 280851 compounds
[Noun Extractor] postprocessing detaching_features : 249805 -> 199708
[Noun Extractor] postprocessing ignore_features : 199708 -> 199316
[Noun Extractor] postprocessing ignore_NJ : 199316 -> 196316
[Noun Extractor] 196316 nouns (280851 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.316 Gb                    
[Noun Extractor] 71.68 % eojeols are covered


In [4]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글
0,2022-07-11 23:42,요즘 한전 게시판 조용하네!ㅋ,2,0,0,요즘 한전 게시판 조용하네,"[요즘, 한전, 게시판, 조용, 하네]"
1,2022-07-11 23:42,1% 올려야 함,4,0,0,올려야 함,"[올려야, 함]"
2,2022-07-11 23:42,=자신들잘못된죄들=모르는=내로남불들=척결...,3,0,0,자신들잘못된죄들 모르는 내로남불들 척결,"[자신들, 잘못된죄들, 모르는, 내로남불, 들, 척결]"
3,2022-07-11 23:42,난 내일 가요,9,0,0,난 내일 가요,"[난, 내일, 가요]"
4,2022-07-11 23:42,실적으로 이만한 실적내는 기업 있으면 나...,6,0,0,실적으로 이만한 실적내는 기업 있으면 나,"[실적, 으로, 이만, 한, 실적, 내는, 기업, 있으면, 나]"
...,...,...,...,...,...,...,...
952447,2020-06-01 00:09,내8자,707,1,3,내 자,"[내, 자]"
952448,2020-06-01 00:08,하락낙폭저지 의 길,682,2,0,하락낙폭저지 의 길,"[하락낙폭저지, 의, 길]"
952449,2020-06-01 00:07,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,697,1,0,주주님들 오랜만에 오르니까 좋네요,"[주주님, 들, 오랜만, 에, 오르니까, 좋네요]"
952450,2020-06-01 00:03,코덱스200선물인버스 곱배기..,725,0,0,코덱스 선물인버스 곱배기,"[코덱스, 선물, 인버스, 곱배기]"


# labeling

---

In [5]:
def labeling(target_df):
    
    f = open("./src/fear_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open("./src/greed_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df


In [6]:
def setting_train_data():
    company = list(pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2)['종목명'][1:])
    dominance = list(pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0]
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김
        train_data = train_data.append(label_df)
    
    return train_data

In [7]:
train_data = setting_train_data()
train_data

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글,label
595805,2021-05-06 20:01,비트코인 대박이네 또 오른다,290,1,1,비트코인 대박이네 또 오른다,"[비트, 코인, 대박, 이네, 또, 오른다]",1
860157,2020-10-06 15:32,내일 모레 실적 발표면 언제가 가장 핫함...,1217,1,1,내일 모레 실적 발표면 언제가 가장 핫함,"[내일, 모레, 실적, 발표, 면, 언제, 가, 가장, 핫함]",-1
29933,2022-07-01 18:53,환율기대 최대 실적,109,1,1,환율기대 최대 실적,"[환율, 기대, 최대, 실적]",-1
910705,2020-07-23 15:22,삼성·SK하이닉스 2분기 실적…기대와 우...,399,0,0,삼성 하이닉스 분기 실적 기대와 우,"[삼성, 하이닉스, 분기, 실적, 기대, 와, 우]",-1
81083,2022-06-20 10:04,공포에 사라,127,0,0,공포에 사라,"[공포, 에, 사라]",-1
...,...,...,...,...,...,...,...,...
10522,2020-11-23 23:54,증권주 하루에 5%씩 오른다,312,4,1,증권주 하루에 씩 오른다,"[증권주, 하루, 에, 씩, 오른다]",1
10377,2020-11-26 20:45,공포의'D'기 온다..삼성이 본사 사옥까...,290,0,2,공포의 기 온다 삼성이 본사 사옥까,"[공포, 의, 기, 온다, 삼성, 이, 본사, 사옥, 까]",-1
3864,2021-06-30 09:11,금리도 실적도,183,0,2,금리도 실적도,"[금리, 도, 실적, 도]",-1
5401,2021-04-23 13:45,"실적M&A 등 상승동력 충분, 증권株 대...",199,3,0,실적 등 상승동력 충분 증권 대,"[실적, 등, 상승동력, 충분, 증권, 대]",-1


In [8]:
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글,label
0,2021-05-06 20:01,비트코인 대박이네 또 오른다,290,1,1,비트코인 대박이네 또 오른다,"[비트, 코인, 대박, 이네, 또, 오른다]",1
1,2020-10-06 15:32,내일 모레 실적 발표면 언제가 가장 핫함...,1217,1,1,내일 모레 실적 발표면 언제가 가장 핫함,"[내일, 모레, 실적, 발표, 면, 언제, 가, 가장, 핫함]",-1
2,2022-07-01 18:53,환율기대 최대 실적,109,1,1,환율기대 최대 실적,"[환율, 기대, 최대, 실적]",-1
3,2020-07-23 15:22,삼성·SK하이닉스 2분기 실적…기대와 우...,399,0,0,삼성 하이닉스 분기 실적 기대와 우,"[삼성, 하이닉스, 분기, 실적, 기대, 와, 우]",-1
4,2022-06-20 10:04,공포에 사라,127,0,0,공포에 사라,"[공포, 에, 사라]",-1
...,...,...,...,...,...,...,...,...
30010,2020-11-23 23:54,증권주 하루에 5%씩 오른다,312,4,1,증권주 하루에 씩 오른다,"[증권주, 하루, 에, 씩, 오른다]",1
30011,2020-11-26 20:45,공포의'D'기 온다..삼성이 본사 사옥까...,290,0,2,공포의 기 온다 삼성이 본사 사옥까,"[공포, 의, 기, 온다, 삼성, 이, 본사, 사옥, 까]",-1
30012,2021-06-30 09:11,금리도 실적도,183,0,2,금리도 실적도,"[금리, 도, 실적, 도]",-1
30013,2021-04-23 13:45,"실적M&A 등 상승동력 충분, 증권株 대...",199,3,0,실적 등 상승동력 충분 증권 대,"[실적, 등, 상승동력, 충분, 증권, 대]",-1
